In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#create a data directory
!mkdir -p /content/data

#extract the main .tar dataset
print("Extracting main dataset (MIO-TCD-Classification.tar)...")
!tar -xf /content/drive/MyDrive/MIO-TCD-Classification.tar -C /content/data

#extract the new .zip file into the 'truck' folder
print("Patching dataset with new images (MIO_new.zip)...")
!unzip /content/drive/MyDrive/MIO_new.zip -d /content/data/train/articulated_truck

print("All files extracted and patched successfully.")

Extracting main dataset (MIO-TCD-Classification.tar)...
Patching dataset with new images (MIO_new.zip)...
Archive:  /content/drive/MyDrive/MIO_new.zip
   creating: /content/data/train/articulated_truck/MIO_new/
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color-[ROI-1]-15.jpg  
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color-[ROI-1]-17.jpg  
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color-[ROI-1]-19.jpg  
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color-[ROI-1]-21.jpg  
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color-[ROI-1]-8.jpg  
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color-_ROI-1_-15.jpg  
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color-_ROI-1_-17.jpg  
  inflating: /content/data/train/articulated_truck/MIO_new/20160927100754-550_color

In [ ]:
import os
import tensorflow as tf

DATA_DIR = '/content/data/train'
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
bad_files = []

print("Starting data integrity scan...")

for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        file_path = os.path.join(root, file)

        # check for non-image files
        if not file.lower().endswith(tuple(image_extensions)):
            print(f"Removing non-image file: {file_path}")
            bad_files.append(file_path)
            os.remove(file_path)
            continue 

        # check for corrupted image files
        try:
            img_bytes = tf.io.read_file(file_path)
            tf.image.decode_image(img_bytes)
        except Exception as e:
            print(f"Removing corrupted image: {file_path}")
            bad_files.append(file_path)
            os.remove(file_path)

print("-" * 30)
print(f"Data integrity check complete. Removed {len(bad_files)} bad files.")

Starting data integrity scan...
------------------------------
Data integrity check complete. Removed 0 bad files.


In [ ]:
import tensorflow as tf

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 11
DATA_DIR = '/content/data/train'

# create training dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# create validation dataset
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# optimize
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

print("Clean datasets are loaded and ready.")

Found 520554 files belonging to 11 classes.
Using 416444 files for training.
Found 520554 files belonging to 11 classes.
Using 104110 files for validation.
Clean datasets are loaded and ready.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2

IMAGE_SHAPE = (224, 224, 3)
NUM_CLASSES = 11

#base model
base_model = MobileNetV2(
    input_shape=IMAGE_SHAPE,
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

#full model
inputs = tf.keras.Input(shape=IMAGE_SHAPE)
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs, outputs)

# load the saved weights fromcheckpoint
print("Architecture built. Loading saved weights from Phase 1...")
model_path = '/content/drive/MyDrive/mio_tcd_classifier_phase1.h5'
model.load_weights(model_path)
print("Weights loaded successfully.")

#unfreeze the base layer for fine-tuning
base_model.trainable = True
print("Base layer is now UN-FROZEN and ready for fine-tuning.")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Architecture built. Loading saved weights from Phase 1...
Weights loaded successfully.
Base layer is now UN-FROZEN and ready for fine-tuning.


In [ ]:
# compile the model for fine-tuning
print("Compiling model for fine-tuning (Phase 2)...")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Low learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# fine-tuning training
print("Starting Phase 2 training...")
history_fine = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10 
)
print("Phase 2 fine-tuning complete.")

# save final model
final_save_path = '/content/drive/MyDrive/mio_tcd_classifier_final.h5'
print(f"Saving Final model to: {final_save_path}")
model.save(final_save_path)

final_keras_path = '/content/drive/MyDrive/mio_tcd_classifier_final.keras'
print(f"Saving Final model to new .keras format: {final_keras_path}")
model.save(final_keras_path)

print("--- ALL DONE! Final model saved. ---")

Compiling model for fine-tuning (Phase 2)...
Starting Phase 2 training...
Epoch 1/10
13014/13014 ━━━━━━━━━━━━━━━━━━━━ 1514s 112ms/step - accuracy: 0.8447 - loss: 0.5135 - val_accuracy: 0.9396 - val_loss: 0.1835
Epoch 2/10
13014/13014 ━━━━━━━━━━━━━━━━━━━━ 1430s 110ms/step - accuracy: 0.9379 - loss: 0.1878 - val_accuracy: 0.9544 - val_loss: 0.1400
Epoch 3/10
13014/13014 ━━━━━━━━━━━━━━━━━━━━ 1446s 109ms/step - accuracy: 0.9557 - loss: 0.1331 - val_accuracy: 0.9576 - val_loss: 0.1313
Epoch 4/10
13014/13014 ━━━━━━━━━━━━━━━━━━━━ 1476s 110ms/step - accuracy: 0.9653 - loss: 0.1039 - val_accuracy: 0.9657 - val_loss: 0.1077
Epoch 5/10
13014/13014 ━━━━━━━━━━━━━━━━━━━━ 1476s 111ms/step - accuracy: 0.9717 - loss: 0.0840 - val_accuracy: 0.9671 - val_loss: 0.1048
Epoch 6/10
13014/13014 ━━━━━━━━━━━━━━━━━━━━ 1436s 110ms/step - accuracy: 0.9767 - loss: 0.0688 - val_accuracy: 0.9689 - val_loss: 0.1040
Epoch 7/10
13014/13014 ━━━━━━━━━━━━━━━━━━━━ 1485s 112ms/step - accuracy: 0.9807 - loss: 0.0574 - val_acc

Phase 2 fine-tuning complete.
Saving Final model to: /content/drive/MyDrive/mio_tcd_classifier_final.h5
Saving Final model to new .keras format: /content/drive/MyDrive/mio_tcd_classifier_final.keras
--- ALL DONE! Final model saved. ---
